In [1]:
from datasets.CSVDataset import CSVDataset
from utils import reduce_dataset
from train import kfold_grid_search
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset = CSVDataset("data/features/Resnet50Model_features.csv")
dataset = reduce_dataset(dataset, discard_ratio=0.0)
print(f"{len(dataset):,} examples")
kfold_grid_search(dataset, in_features=2048, device=device,
                  checkpoint_file_path=None,  #"grid-search-25.json",
                  batch_size=256,
                  max_epochs=20,
                  hidden_layer_combs=[1,2,3],
                  unit_combs=[512,1024,2048],
                  learning_rate_combs=[0.0001],
                  dropout_combs=[0.2,0.3,0.4],
                  threshold_combs=[0.5],
                  focal_alpha_combs=[0.02, 0.03, 0.04],
                  )


data/features/Resnet50Model_features.pickle
20,227 examples
(0/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.02, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 576.88it/s]


{'test_loss': 0.0024414504165179095, 'test_accuracy': 0.8103867400248639, 'test_precision': 0.19997811324149609, 'test_recall': 0.8377189468107871, 'test_f1': 0.3200251275056835, 'test_mcc': 0.3475303624135175}
(1/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.03, focal_gamma=2.0)


Testing: 100%|██████████| 16/16 [00:00<00:00, 541.05it/s]


{'test_loss': 0.00391716617741622, 'test_accuracy': 0.798330633121448, 'test_precision': 0.19251235133383346, 'test_recall': 0.851451332143089, 'test_f1': 0.3110738354677496, 'test_mcc': 0.34098709568661806}
(2/81) (hidden_layers=1, units=512, dropout=0.2, threshold=0.5, learning_rate=0.0001, weight_decay=0.0, focal_alpha=0.04, focal_gamma=2.0)


Testing:  94%|█████████▍| 15/16 [00:00<00:00, 526.43it/s]

KeyboardInterrupt

